# **Twitter API:**

Using tweepy, which is an easy-to-use Python library for accessing the Twitter API



### **Setting up the environment**

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Installing requirements

In [4]:
%%capture
!pip install tweepy

###**Importing the required modules**

In [5]:
# import relevant libraries
import tweepy
import pandas as pd
import numpy as np
import os
import csv

### **Connecting to twitter**

In [6]:
# load twitter API secrets from an external file
config = pd.read_csv("/content/drive/MyDrive/Twitter Analysis/config.csv")

twitterApiKey = config['twitterApiKey'][0]
twitterApiSecret = config['twitterApiSecret'][0]
twitterApiAccessToken = config['twitterApiAccessToken'][0]
twitterApiAccessTokenSecret = config['twitterApiAccessTokenSecret'][0]

### **Extracting replies after Biden's first 100 days of presidency**

In [ ]:
# function to get all replies for a specified tweet
def get_replies(name, tweet_id, count):
  '''
  input:
    name: a chosen hashtag
    tweet_id: tweet id from where to process the replies (can be found within the tweet URL)
    count: the number of tweets to extract
  output:
    csv: a csv file containing time and text
  '''  
  # authorize access
  auth = tweepy.OAuthHandler(twitterApiKey, twitterApiSecret)
  auth.set_access_token(twitterApiAccessToken, twitterApiAccessTokenSecret)
  api = tweepy.API(auth, wait_on_rate_limit=True)

  # initialize list and fetch tweets
  replies = []
  for tweet in tweepy.Cursor(api.search,q='to:'+name, tweet_mode="extended", result_type='recent', timeout=999999).items(count):
    if hasattr(tweet, 'in_reply_to_status_id_str'):
      if (tweet.in_reply_to_status_id_str==tweet_id):
        replies.append(tweet)

  # write to csv
  with open(f'/content/drive/MyDrive/Twitter Analysis/data/{name}_replies.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=('date', 'user', 'text'))
    writer.writeheader()
    for tweet in replies:
      row = {'date': tweet.created_at, 'user': tweet.user.screen_name, 'text': tweet.full_text}
      writer.writerow(row)
  pass 

In [ ]:
# call the function to get the most recent replies to Biden's tweet regarding his first 100 days of presidency
%%capture
get_replies('JoeBiden', '1387800729281236997', 30000)

### **Extracting recent tweets from Joe Biden's twitter account**

In [14]:
# function to get all tweets from as specific user
def get_all_tweets(username, count):
  '''
  input:
    name: a chosen hashtag
    tweet_id: tweet id from where to process the replies (can be found within the tweet URL)
    count: the number of tweets to extract
  output:
    csv: a csv file containing time and text
  '''  
  # authorize access
  auth = tweepy.OAuthHandler(twitterApiKey, twitterApiSecret)
  auth.set_access_token(twitterApiAccessToken, twitterApiAccessTokenSecret)
  api = tweepy.API(auth, wait_on_rate_limit=True)

  # initialize list and fetch tweets
  replies = []
  for tweet in tweepy.Cursor(api.user_timeline, screen_name = username, tweet_mode="extended", result_type='recent', timeout=999999).items(count):
    replies.append(tweet)

  # write to csv
  with open(f'/content/drive/MyDrive/Twitter Analysis/data/{username}_tweets.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=('date', 'user', 'text', 'retweets', 'favourites'))
    writer.writeheader()
    for tweet in replies:
      row = {'date': tweet.created_at, 'user': tweet.user.screen_name, 'text': tweet.full_text, 'retweets': tweet.retweet_count,'favourites': tweet.favorite_count}
      writer.writerow(row)
  pass 

In [15]:
# call the function to get the most recent tweets of Joe Biden
# the maximum tweets allowed is ~3200
%%capture
get_all_tweets('@JoeBiden', 30000)